# 事前構築されたカスタムAIサービスの統合

このノートブックでは、Computer Vision APIおよびText Analytics APIと統合して、クレーム処理機能を強化します。 最後に、展開したサマライザサービスと分類子サービスへのAPI呼び出しを統合し、クレームテキストとクレームイメージに適用されたすべての処理を示す完成したクレームレポートを作成します。

## Task 1 - Computer Vision APIを使用したキャプションとタグ

下のセルで、コンピュータービジョンAPIのキーを指定してセルを実行します。

In [ ]:
subscription_key = '' #"<your_computer_vision_api_key>"
assert subscription_key

次のセルを実行して、Computer Vision APIのエンドポイントを作成します。 最後のパスセグメントはanalyzeです。これは、分析機能を使用することを示しています。

Computer VisionサービスのインスタンスのAzure Portalからコピーしたエンドポイント値と一致するように、以下の vision_endpoint の値を必ず更新してください。 値がスラッシュ（/）で終わっていることを確認してください

In [ ]:
vision_endpoint = '' #"<your_computer_vision_api_endpoint>"
vision_base_url = vision_endpoint + "vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze"

次のセルには、単純なWeb検索を実行した後に見つかったサンプル画像のリストが含まれています。 選択した画像のURLを自由に置き換えてください。

In [ ]:
fender_bender = "https://www.washingtonpost.com/blogs/innovations/files/2015/02/Stolen_Car_Crash-00aef.jpg"
damaged_house = "https://c2.staticflickr.com/8/7342/10983313185_0589b74946_z.jpg"
police_car = "https://localtvwnep.files.wordpress.com/2015/11/fender-bender.jpeg"
car_with_text = "https://static.buildasign.com/cmsimages/bas-vinyl-lettering-splash-01.png"
car_tow = 'https://i.ytimg.com/vi/wmxJ2FrzTWo/maxresdefault.jpg'

上記の画像のリストから1つを選択し、それを image_url に割り当ててさらに処理します。

In [ ]:
image_url = car_tow

次のセルを実行して、選択した画像をプレビューします。

In [ ]:
from IPython.display import Image, display
display(Image(image_url))

次のセルは、Computer Vision APIに対して行うHTTPリクエストを作成します。

次のセルを実行して、キャプションとタグを取得します。

In [ ]:
import requests
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {'visualFeatures': 'Categories,Description,Tags,Color'}
data     = {'url': image_url}
response = requests.post(vision_analyze_url, headers=headers, params=params, json=data)
response.raise_for_status()
analysis = response.json()
analysis

上記の出力からわかるように、結果はネストされたドキュメント構造です。 次のセルを実行して、キャプションと上位3つのタグの結果を引き出します。

In [ ]:
caption = analysis["description"]["captions"][0]["text"].capitalize()
caption

In [ ]:
topTags = analysis["description"]["tags"][0:3]
topTags

## Task 2 - OCRの実行

Computer VisionサービスでOCRを実行するには、OCRエンドポイントをターゲットにする必要があります。

次のセルを実行して、正しいURLを作成します。

In [ ]:
vision_ocr_url = vision_base_url + "ocr"

次に、次のコードでOCRエンドポイントを呼び出し、結果を調べます。

In [ ]:
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {}
data     = {'url': image_url}
response = requests.post(vision_ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()
ocr_analysis = response.json()
ocr_analysis

結果からテキストをフラット配列として抽出するための次のコードを提供します。

次のセルを実行して、結果のドキュメントからテキストアイテムを抽出します。

In [ ]:
import itertools
flatten = lambda x: list(itertools.chain.from_iterable(x))
words_list = [[ [w['text'] for w in line['words']]  for line in d['lines']] for d in ocr_analysis['regions']]
words_list = flatten(flatten(words_list))
print(list(words_list))

## Task 3 - 感情分析の実行

感情分析は、Text Analytics APIを使用して実行されます。

次のセルをText Analytics APIのインスタンスのキーで更新し、セルを実行します。

In [ ]:
text_analytics_subscription_key = '' #"<your_text_analytics_key>"
assert text_analytics_subscription_key

Text Analytics APIのデプロイされたインスタンスの正しいベースURLで次のセルを更新し、セルを実行します。

In [ ]:
#"<your_text_analytics_base_url>"
text_analytics_base_url = ''
sentiment_api_url = text_analytics_base_url + "/text/analytics/v2.1/sentiment"

次のセルには、測定の感情をテストするために使用できる一連のクレーム例があります。

セルを実行します。

In [ ]:
neg_sent = """We are just devastated and emotionally drained. 
The roof was torn off of our car, and to make matters
worse my daughter's favorite teddy bear was impaled on the street lamp."""
pos_sent = """We are just happy the damaage was mininmal and that everyone is safe. 
We are thankful for your support."""
neutral_sent = """I crashed my car."""
long_claim = """
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon. The sun was bright and shining just behind the stoplight.
This made it hard to see the lights. There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red. After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. The license plate of the black sedan was ABC123. 
"""

上記のクレームのリストから1つを選択し、その変数を claim_text に割り当てて、Text Analytics APIの呼び出しで使用します。

In [ ]:
claim_text = long_claim

APIでは、次のフォームのドキュメントを送信する必要があります。

セルを実行してリクエストドキュメントを作成します。

In [ ]:
documents = {'documents' : [
    {'id': '1', 'language': 'en', 'text': claim_text}
]}

次に、Text Analytics APIを呼び出して、結果を確認します。

In [ ]:
headers   = {"Ocp-Apim-Subscription-Key": text_analytics_subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiments = response.json()
sentiments

レスポンスからの感情スコアをパースするには、次のセルを実行します。

In [ ]:
score = sentiments['documents'][0]['score']
score

次のセルを実行することにより、このスコアについて人間にわかりやすい解釈を提供できます。

In [ ]:
score_interpretation = "neutral"
if (score < 0.45): 
    score_interpretation = "negative"
elif (score >= 0.55):
    score_interpretation = "positive"
score_interpretation

## Task 4 - Azure MLデプロイ済みサービスの呼び出し

次のセルを実行して、Azure Machine Learningサービスを使用してAzure Container Instancesにデプロイされた分類メソッドと要約メソッドを呼び出すために使用されるメソッドを定義します。

In [ ]:
def invoke_service(ml_service_key, ml_service_scoring_endpoint, ml_service_input):
    headers   = {"Authorization": "Bearer " + ml_service_key}
    response  = requests.post(ml_service_scoring_endpoint, headers=headers, json=ml_service_input)
    result = response.json()
    return result

デプロイされたインスタンスに応じて、キーとエンドポイントを使用して分類の呼び出しを構成します。

In [ ]:
classifier_service_key = "" #サービスで認証が有効になっている場合のみKeyを入力(今回は不要)

#"<your_classifier_scoring_url>" エンドポイントのURLを入力
classifier_service_scoring_endpoint = ''
classifier_service_input = [claim_text]

分類を呼び出して結果を確認します。

In [ ]:
classifier_result = invoke_service(classifier_service_key, 
                                   classifier_service_scoring_endpoint, classifier_service_input)
classifier_result

In [ ]:
# Interpret the classifier result
classification = 'Auto Insurance Claim' if classifier_result == 1 else 'Home Insurance Claim' 
classification

同様に、サマライザサービスに応じて、キーとスコアリングエンドポイントを構成します。

In [ ]:
summarizer_service_key = "" #サービスで認証が有効になっている場合のみKeyを入力(今回は不要)

#"<your_summarizer_service_url>" エンドポイントのURLを入力
summarizer_service_scoring_endpoint = ''
summarizer_service_input = claim_text

サマライザサービスを呼び出し、結果を確認します。

In [ ]:
summarizer_result = invoke_service(summarizer_service_key, summarizer_service_scoring_endpoint, 
                                   summarizer_service_input)
formatted_result =  summarizer_result[0].replace("\\n", " ").strip() if len(summarizer_result) > 0 else "N/A"
formatted_result

## Task 5 - 結果の統合

この最後のタスクでは、すべての要素をまとめて、AIベースの処理の結果を表示します。

次のセルを実行し、結果を調べます。

In [ ]:
from IPython.core.display import HTML

displayTemplate = """
<div><b>Claim Summary</b></div>
<div>Classification: {}</div>
<div>Caption: {}</div>
<div>Tags: {}</div>
<div>Text in Image: {}</div>
<div>Sentiment: {}</div>
<div><img src='{}' width='200px'></div>
<div>Summary: </div>
<div><pre>{} </pre></div>
<div>&nbsp;</div>
<div>Claim:</div>
<div>{}</div>

"""
displayTemplate = displayTemplate.format(classification, caption, ' '.join(topTags), ' '.join(words_list), 
                                         score_interpretation, image_url, formatted_result, 
                                         claim_text)
display(HTML(displayTemplate))